In [131]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import plotly.express as px
import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr

In [132]:
wr.config.s3_endpoint_url = "http://192.168.1.7:8333"


In [133]:

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)

In [134]:
multilabelDf

,vision_bonnet,vision_bumper_front,vision_grille,vision_headlamp_rh,vision_headlamp_lh,vision_door_front_lh,vision_door_front_rh,vision_engine,vision_bumper_rear,vision_misc,...,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear,CaseID,Circumstances_of_Accident
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,12657878,Collision- Head to Rear (Insured Hit TP)
1,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,12657888,Lost control- Overturned
2,1,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657890,
3,1,1,1,0,0,0,1,1,0,1,...,1,0,0,0,0,0,0,0,12657900,Collided into animal
4,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657903,Collision- Head to Rear (Insured Hit TP)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621787,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10147236,
621788,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10936139,Collision- Head to Rear (TP Hit Insured)
621789,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174259,
621790,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174498,


In [135]:
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
   
]
allInputFeature = caseFeatures + allVisionFeatures
targetCol = [
    x
    for x in multilabelDf.columns
    if x not in allInputFeature and x != "CaseID"
]
len(targetCol)

34

In [136]:
saloonPredDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/saloon_v1/imgs_pred_output.csv")
hatchBackDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/hatchback_v1/Hatchback - 5 Dr_imgs_pred_output.csv")
imgPredOutput = pd.concat([saloonPredDf, hatchBackDf])

In [137]:
realTestDataDf = multilabelDf[caseFeatures + targetCol + ["CaseID"]].merge(imgPredOutput, on="CaseID")
realTestDataDf = realTestDataDf.loc[:, ~realTestDataDf.columns.str.contains('^Unnamed')]


In [138]:
realTestDataDf.to_csv("saloon_real_test.csv")

In [139]:
trainDf = multilabelDf[~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())]
assert set(realTestDataDf['CaseID'].tolist()).isdisjoint(trainDf['CaseID'].tolist())

In [140]:
# X_train = trainDf[allInputFeature]
# Y_train = trainDf[targetCol]
X = realTestDataDf[allInputFeature]
Y = realTestDataDf[targetCol]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

train_pool = Pool(X_train, Y_train, cat_features=caseFeatures + allVisionFeatures)
test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)

In [141]:
clf = CatBoostClassifier(
    loss_function="MultiCrossEntropy",
    eval_metric="Accuracy",
    iterations=100,
    # task_type="GPU",
    class_names=targetCol,
)
clf.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=10)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.200546
0:	learn: 0.0109503	test: 0.0114432	best: 0.0114432 (0)	total: 353ms	remaining: 35s
10:	learn: 0.0277211	test: 0.0292986	best: 0.0292986 (10)	total: 3.29s	remaining: 26.7s
20:	learn: 0.0334429	test: 0.0342310	best: 0.0342310 (20)	total: 6.26s	remaining: 23.6s
30:	learn: 0.0348570	test: 0.0345270	best: 0.0345270 (30)	total: 9.1s	remaining: 20.3s
40:	learn: 0.0363038	test: 0.0353162	best: 0.0353162 (40)	total: 11.9s	remaining: 17.2s
50:	learn: 0.0380138	test: 0.0366972	best: 0.0366972 (50)	total: 15.1s	remaining: 14.5s
60:	learn: 0.0388688	test: 0.0375851	best: 0.0375851 (60)	total: 18.1s	remaining: 11.6s
70:	learn: 0.0410720	test: 0.0376837	best: 0.0376837 (70)	total: 21.1s	remaining: 8.6s
80:	learn: 0.0430122	test: 0.0372891	best: 0.0376837 (70)	total: 24.1s	remaining: 5.64s
90:	learn: 0.0440973	test: 0.0374864	best: 0.0376837 (70)	total: 27.1s	remaining: 2.67s
99:	learn: 0.0454127	test: 0.0371905	best: 0.0376837 (70)	total: 29.7s	remaining: 0us

bestTest 

In [142]:
test_predict = clf.predict(X_test)
train_predict = clf.predict(X_train)

In [143]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list), acc_list

In [144]:
accuracy = eval_metric(Y_test.values, test_predict, "Accuracy")[0]
print(f"Accuracy: {accuracy}")

Accuracy: 0.03768373285982046


In [145]:
subsetAcc2, acc_list = hamming_score(Y_test.values, test_predict)
subsetAcc2

0.513721731123999

In [146]:
avgPrecision = 0
avgRecall = 0
avgF1 = 0
allRecall = []
allPrecision = []
clsOrder = clf.classes_
allF1 = []
allSupport = []
allAcc = []
allBalanceAcc = []
highRecallCount = 0
lowRecallCount = 0
recallLowThreshold = 0.2
highRecallThreshold = 0.6
for metric in ("Precision", "Recall", "F1", "Accuracy"):
    # print(metric)
    values = eval_metric(Y_test.values, test_predict, metric)
    if metric == "Precision":
        avgPrecision = np.mean(values)
    elif metric == "Recall":
        avgRecall = np.mean(values)

    elif metric == "F1":
        avgF1 = np.mean(values)
    for cls, value in zip(clf.classes_, values):
        
        # print(f"class={cls}: {value:.4f}")
        if metric == "Recall":
            allRecall.append(value)
            if value <= recallLowThreshold:
                lowRecallCount += 1
            elif value >= highRecallThreshold:
                highRecallCount += 1
        elif(metric == "Precision"):
            allPrecision.append(value)
        elif(metric == "F1"):
            allF1.append(value)
        elif metric == "Accuracy":
            allAcc.append(value)
    print()
print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Low Recall Count : {lowRecallCount}")
print(f"High Recall Count : {highRecallCount}")
print(f"all acc : {allAcc}")

summaryPerfDf = pd.DataFrame({
    "recall" : allRecall,
    "precision" : allPrecision,
    "f1" : allF1,
    "part" :  clsOrder,
    "avgRecall" : avgRecall,
    "avgPrecision" : avgPrecision,
    "avgF1" : avgF1,


})





Avg Precision : 0.6639490408135647
Avg Recall : 0.4196833606905915
Avg F1 : 0.478214976189253
Low Recall Count : 9
High Recall Count : 11
all acc : [0.03768373285982046]


In [147]:
avgDf = pd.DataFrame(
    {
        "AvgRecall" : [avgRecall],
        "AvgPrecision" : [avgPrecision],
        "AvgF1" : [avgF1],
        "ExactMatch" : [accuracy],
        "SubsetAccuracy" : [subsetAcc2],


    }
).transpose().reset_index()
# fig = px.bar(avgDf, x = 'part', y = 'recall')
fig = px.bar(avgDf, x = 'index', y = 0, text_auto=True)

fig

In [148]:
fig = px.bar(summaryPerfDf, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 1), 0.2, 0.5, 0.7],
    )
)
fig

In [149]:
fig = px.bar(summaryPerfDf, x = 'part', y = 'precision')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 1), 0.2, 0.5, 0.7],
    )
)
fig

In [150]:
fig = px.bar(summaryPerfDf, x = 'part', y = 'f1')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 1), 0.2, 0.5, 0.7],
    )
)
fig